# Web Search

> No sailboat can move itself by its own wind; it needs wind from outside. -- Yin Keming [*Quantum Elegy*](https://read.douban.com/ebook/12945743/)

In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.tools import tool

# Load model configuration
_ = load_dotenv()

# Load model
llm = ChatOpenAI(
    model="qwen-max",
    temperature=0.7,
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url=os.getenv("DASHSCOPE_BASE_URL"),
)

In [2]:
# !pip install dashscope langchain-community tavily-python ddgs

## 1. Using DashScope

First install dashscope:

```bash
pip install dashscope
```

You only need to set the network parameter `enable_search` of the `Generation.call` function to `True` to access the internet. Below we make this function into a tool for the Agent to call.

In [3]:
from dashscope import Generation

@tool
def dashscope_search(query: str) -> str:
    """
    Search the internet using Quark Search API.
    """
    response = Generation.call(
        model='qwen3-max',
        prompt=query,
        enable_search=True,
        result_format='message'
    )

    if response.status_code == 200:
        return response.output.choices[0].message.content
    else:
        return (
            "Search failed with status code: "
            f"{response.status_code}, message: {response.message}"
        )

# Create Agent
agent = create_agent(
    model=llm,
    tools=[dashscope_search],
    system_prompt="You are an intelligent assistant. You must use tools to search the internet before answering.",
)

# Run Agent
response = agent.invoke(
    {"messages": [{
        "role": "user",
        "content": "Tell me today's date and the most important news of today"
    }]}
)

In [4]:
# Get all Agent responses
for message in response['messages']:
    message.pretty_print()

================================ Human Message =================================

Tell me today's date and the most important news of today
================================== Ai Message ==================================
Tool Calls:
  dashscope_search (call_f053c2e0ef38486ebc63fc)
 Call ID: call_f053c2e0ef38486ebc63fc
  Args:
    query: today's date
  dashscope_search (call_031c2cca4266400d838b7a)
 Call ID: call_031c2cca4266400d838b7a
  Args:
    query: most important news today
================================= Tool Message =================================
Name: dashscope_search

Today's date is June 27, 2024.
================================= Tool Message =================================
Name: dashscope_search

I can't provide real-time news updates, but you can check trusted sources like:

- **BBC News** (bbc.com/news)  
- **Reuters** (reuters.com)  
- **The New York Times** (nytimes.com)  
- **Associated Press** (apnews.com)  
- **Al Jazeera** (aljazeera.com)

For the most import

## 2. Using Tavily

First install tavily:

```bash
pip install langchain-community tavily-python
```

Then apply for an `API_KEY` from the [Tavily](https://www.tavily.com/) official website and fill it in the box below.

In [5]:
import getpass
os.environ["TAVILY_API_KEY"] = getpass.getpass()

 ········


In [6]:
from langchain_community.retrievers import TavilySearchAPIRetriever

retriever = TavilySearchAPIRetriever(k=3)
query = "Tell me today's date and the most important news of today"

The returned content is a bit much. You can uncomment the command below to view it 👇

In [7]:
# retriever.invoke(query)

Similarly, let's add Tavily as a tool to the Agent and see how it works!

In [8]:
prompt_template = """Answer the question based on the search results below:

Search Results: {context}

Question: {question}"""

def format_docs(docs):
    """Merge Tavily search results"""
    return "\n\n".join(doc.page_content for doc in docs)

@tool
def tavily_search(query: str) -> str:
    """
    Search the internet using Tavily Search API.
    """
    retriever = TavilySearchAPIRetriever(k=3)
    contents = retriever.invoke(query)
    if len(contents) > 0:
        return format_docs(contents)
    else:
        return "No search results"

# Create Agent
agent = create_agent(
    model=llm,
    tools=[tavily_search],
    system_prompt="You are an intelligent assistant. You must use tools to search the internet before answering.",
)

# Run Agent
response = agent.invoke(
    {"messages": [{
        "role": "user",
        "content": "Tell me today's date and the most important news of today"
    }]}
)

In [9]:
# Get all Agent responses
for message in response['messages']:
    message.pretty_print()

================================ Human Message =================================

Tell me today's date and the most important news of today
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_1acff9a49f1042fdb92a36)
 Call ID: call_1acff9a49f1042fdb92a36
  Args:
    query: today's date
  tavily_search (call_97be4e504a0e4142bac235)
 Call ID: call_97be4e504a0e4142bac235
  Args:
    query: most important news of today
================================= Tool Message =================================
Name: tavily_search

Today is: ; Gregorian: Friday, 13 February 2026 ; Mayan: Long count = 13.0.13.6.2; tzolkin = 9 Ik; haab = 0 Kayab ; French: 25 Pluviôse an 234 de la Révolution.

Details about today's date with count of days, weeks, and months, Sun and Moon cycles, Zodiac signs and holidays.

Today, February 12th , is day 43 of 365 total days in 2026. What is Today's Date in Numbers? Today's date in numbers is: MM-DD-YYYY: 02-12-2

## 3. Using DDGS

First install DDGS:

```bash
pip install ddgs
```

We directly use the summary information obtained from DDGS without further web crawling. Below we make DDGS into a tool.

In [10]:
from ddgs import DDGS

# Create ddgs client
ddgs = DDGS()

@tool
def ddgs_search(query: str) -> str:
    """
    Search the internet using DDGS Search API.

    Args:
        query: Search keywords or questions.
        max_results: Maximum number of results to return.

    Returns:
        A string containing the title, summary, and link for each search result.
    """
    results = list(
        ddgs.text(
            query=query,
            region="wt-wt",  # wt-wt zh
            timelimit='y',
            safesearch='off',  # moderate off
            page=1,
            backend='auto',
            max_results=3,
        )
    )

    content = ""
    for i, r in enumerate(results, 1):
        content += f"[Result {i}]\n"
        content += f"Title: {r['title']}\n"
        content += f"Summary: {r['body']}\n"
        content += f"Link: {r['href']}\n\n"

    return content

# Create Agent
agent = create_agent(
    model=llm,
    tools=[ddgs_search],
    system_prompt="You are an intelligent assistant. You must use tools to search the internet before answering.",
)

# Run Agent
response = agent.invoke(
    {"messages": [{
        "role": "user",
        "content": "Tell me today's date and the most important news of today"
    }]}
)

In [11]:
# Get all Agent responses
for message in response['messages']:
    message.pretty_print()

================================ Human Message =================================

Tell me today's date and the most important news of today
================================== Ai Message ==================================
Tool Calls:
  ddgs_search (call_1f195153bd6b418aaae7a6)
 Call ID: call_1f195153bd6b418aaae7a6
  Args:
    query: today's date
    max_results: 1
  ddgs_search (call_c7a98f050a784f198751b4)
 Call ID: call_c7a98f050a784f198751b4
  Args:
    query: most important news today
    max_results: 1
================================= Tool Message =================================
Name: ddgs_search

[Result 1]
Title: Today's Date - Find Out Quickly What's The Date Today ❤️
Summary: Throw the first stone who has never been lost in time after a big hangover and forget what is the date today . ... work and want to know today ' s date ...
Link: https://calendarhours.com/todays-date/

[Result 2]
Title: What is the date today | Today's Date
Summary: Today ' s Date is your one-stop desti